In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import math
import glob
import pickle
import statistics
import scipy.stats as stats
from sklearn.neighbors import KernelDensity
import dask
import seaborn as sns
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [2]:
def get_files():
    models = glob.glob("/terra/data/cmip5/global/historical/*")
    avail={}
    for model in models:
        zg = glob.glob(str(model)+"/r1i1p1/day/2deg/zg*")
        try:
            test = zg[0]
            avail[model.split('/')[-1]] = zg
        except:
             pass
    return avail

In [3]:
files = get_files()

In [4]:
files['NOAA'] = glob.glob("/home/pmarsh/NOAA_2deg/NOAA_zg/*.nc")
files['ERA5'] = glob.glob("/home/pmarsh/NOAA_2deg/ERA5_zg/*.nc")
files.pop('MIROC-ESM')

['/terra/data/cmip5/global/historical/MIROC-ESM/r1i1p1/day/2deg/zg_day_MIROC-ESM_historical_r1i1p1_19500101-20051231.nc']

In [5]:
def contourise(x):
    x = x.fillna(0)
    x = x.where((x>=limit))
    x = x/x
    return x

In [ ]:
results={}
for model in files.keys():
    print(model)
    x = xr.open_mfdataset(files[model])
    if model == 'NOAA':
        x = x.rename({'hgt':'zg'})
        x = x.rename({'level':'plev'})
        x = x.sel(plev=850)
        x = x.sel(time=slice('1950','2005'))
    elif model == 'ERA5':
        x = x.rename({'level':'plev'})
        x = x.sel(plev=850)
        x = x.sel(time=slice('1979','2005'))
    else:
        x = x.sel(plev=85000)
        x = x.sel(time=slice('1950','2005'))
        x = x.load()
    x = x.sel(lat=slice(-60,0))
    x = x[['zg']]
    x = x.assign_coords(lon=(((x.lon + 180) % 360) - 180))
    with dask.config.set(**{'array.slicing.split_large_chunks': True}):
        x = x.sortby(x.lon)
    x = x.sel(lon=slice(-50,20))
    x = x.resample(time="QS-DEC").mean(dim="time",skipna=True)
    x = x.load()
    limit = np.nanquantile(x.zg.values,0.9)
    results[model]={}
    for seas in ['DJF','MAM','JJA','SON']:
        mean_seas = x.where(x.time.dt.season==str(seas)).dropna(dim='time')
        mean_seas = contourise(mean_seas).zg.fillna(0).mean(dim='time')
        results[model][seas] = mean_seas.fillna(0)
    x.close()

IPSL-CM5A-LR


In [ ]:
pickle.dump(results, open( "../HIGH_OUT/SASH_track_2D.p", "wb" ) )

In [ ]:
weights = np.cos(np.deg2rad(results['NOAA']['DJF'].lat)) #area weighted

In [ ]:
#mean absolute error calc
scores=[]
for index in results:
    MAE={}
    for season in ['DJF','MAM','JJA','SON']:
        ref = results['NOAA'][season]
        x = results[index][season]
        MAE[season] = (np.abs(ref - x)).weighted(weights).sum(('lat','lon'))
    scores.append([index,np.mean(MAE['DJF'].values + MAE['MAM'].values + MAE['JJA'].values + MAE['SON'].values)])

In [ ]:
resultsdf = pd.DataFrame(np.array(scores),columns=['model','score'])
resultsdf = resultsdf.sort_values('score').set_index('model')['score']

In [ ]:
pickle.dump( resultsdf, open( "../HIGH_OUT/scores_2D.p", "wb" ) )
resultsdf.to_csv("../HIGH_OUT/scores_2D.csv")